In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), download_data=False, verbose=False)

Setting up Sandbox...
Done!


In [10]:
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = th.abs(x)
    return x

In [11]:
plan_double_abs.build(th.tensor([1., -2.]))

In [12]:
sy.serde._simplify(plan_double_abs)

(18,
 ([[6,
    [1,
     [6,
      [[6,
        [[5, [b'__add__']],
         [19, [77595072276, 66733553180, 39084299439, None, [13, [2]], False]],
         [6,
          [[19,
            [77595072276, 66733553180, 39084299439, None, [13, [2]], False]]]],
         [0, []]]],
       [1, [27020424311]]]]]],
   [6,
    [1,
     [6,
      [[6,
        [[5, [b'torch.abs']],
         None,
         [6,
          [[19, [78344051041, 27020424311, 39084299439, None, None, True]]]],
         [0, []]]],
       [1, [53782158465]]]]]],
   [6, [9, 27020424311]]],
  39084299439,
  (1, [66733553180]),
  (1, [53782158465]),
  (5, (b'plan_double_abs',)),
  None,
  None,
  True))